In [671]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [672]:
class DSO:
	def __init__(self, totalHours, marginalCost, unitSalePrice, maxPriceReference):
		self.totalHours = totalHours
		self.numberOfAggregators = 0
		self.marginalCost = marginalCost
		self.unitSalePrice = unitSalePrice
		self.maxPriceReference = maxPriceReference
		self.aggregators = []
		self.maxUtility = 0
		self.m = 0

	def addAggregator(self, aggregator):
		self.aggregators.append(aggregator)
		self.numberOfAggregators = self.numberOfAggregators + 1

	def getAggregators(self):
		return self.aggregators

	def Sfunction(self, aggregator, tIndex):
		return aggregator.maximumDemand[tIndex] * aggregator.maxPriceReference * (1 - np.exp(-aggregator.preferenceSatisfaction[tIndex] * (aggregator.bestLoadResponse[tIndex] / aggregator.nominalDemand[tIndex])))
	
	def CostFunction(self, aggregator, tIndex):
		return self.marginalCost[tIndex]
	
	def getLeftLimit(self):
		leftLimit = [0 for i in range(self.totalHours)]
		for i in range(self.totalHours):
			mx = 0
			for j in range(self.numberOfAggregators):
				agg = self.aggregators[j]
				mx = max(mx, agg.preferenceSatisfaction[i] * agg.maxPriceReference * np.exp(-agg.preferenceSatisfaction[i] * (agg.maximumDemand[i] / agg.nominalDemand[i])))
			leftLimit[i] = mx
		return leftLimit
	
	def getRightLimit(self):
		rightLimit = [0 for i in range(self.totalHours)]
		for i in range(self.totalHours):
			mx = 0
			for j in range(self.numberOfAggregators):
				agg = self.aggregators[j]
				mx = max(mx, agg.preferenceSatisfaction[i] * agg.maxPriceReference * np.exp(-agg.preferenceSatisfaction[i] * (agg.minimumDemand[i] / agg.nominalDemand[i])))
			rightLimit[i] = mx
		return rightLimit
	
	def generateUnitSalePrice(self):
		minPrice = self.marginalCost
		leftLimit = self.getLeftLimit()
		minPrice = np.minimum(minPrice, leftLimit)

		maxPrice = self.maxPriceReference
		rightLimit = self.getRightLimit()
		maxPrice = np.maximum(maxPrice, rightLimit)

		for i in range(self.totalHours):
			self.unitSalePrice[i] = random.uniform(minPrice[i], maxPrice[i])

	def generateM(self):
		m = 0
		mx = 0
		for i in range(self.totalHours):
			sum = 0
			for j in range(self.numberOfAggregators):
				agg = self.aggregators[j]
				sum = sum + agg.bestLoadResponse[i]
			mx = max(mx, sum)
		m = random.uniform(mx, 100000)
		self.m = m

	def utilityFunction(self, m, theta, omega):
		# self.generateUnitSalePrice()
		self.generateM()

		# store value of INT_MIN in mx
		mx = -1000000000000
		for j in range(self.totalHours):
			sum = 0
			for i in range(self.numberOfAggregators):
				agg = self.aggregators[i]
				sum = sum + self.unitSalePrice[j] * agg.bestLoadResponse[j] - self.CostFunction(agg, j) * agg.bestLoadResponse[j] + omega * self.Sfunction(agg, j)
			sum = sum - theta*m
			mx = max(mx, sum)
		return mx

	def getPAR(self):
		# sum of all bestResponses for each hour for each aggregator
		sum = 0
		for i in range(self.totalHours):
			for j in range(self.numberOfAggregators):
				sum = sum + self.aggregators[j].bestLoadResponse[i]
		return (self.m * self.totalHours) / (sum)

class Aggregator:
	def __init__(self, totalHours, nominalDemand, minimumDemand, maximumDemand, maxPriceReference, preferenceSatisfaction):
		self.totalHours = totalHours
		self.nominalDemand = nominalDemand
		self.minimumDemand = minimumDemand
		self.maximumDemand = maximumDemand
		self.maxPriceReference = maxPriceReference
		self.preferenceSatisfaction = preferenceSatisfaction
		self.currentUtility = 0
		self.bestLoadResponse = []

	def utilityFunction(self, DSO):
		sum = 0
		for i in range(DSO.totalHours):
			sum = sum + DSO.Sfunction(self, i) - (DSO.unitSalePrice[i] * self.bestLoadResponse[i])
		return sum

	def optimalDR(self, DSO):
		self.bestLoadResponse = []
		for j in range(DSO.totalHours):
			self.bestLoadResponse.append((self.nominalDemand[j] / self.preferenceSatisfaction[j]) * (np.log((self.preferenceSatisfaction[j] * self.maxPriceReference) / DSO.unitSalePrice[j])))


In [673]:
datasetPath = '../dataset/'

In [674]:
numOfLA = 3
numOfHours = 500

In [675]:
nominalDemands = []
for i in range (0, 3):
	letter = chr(ord('A') + i)
	fileName = 'Home' + letter + '_hourly.csv'
	df = pd.read_csv(datasetPath + fileName)
	df = df[0:numOfHours]
	nominalDemands.append(df['Power'].tolist())

In [676]:
LAs = []
maxPriceReference = np.random.randint(1, 1000)
for i in range(numOfLA):
	nominalDemand = nominalDemands[i]
	# find max of nominalDemand
	maxNominalDemand = max(nominalDemand)
	minimumDemand = np.random.uniform(0.01, nominalDemand, numOfHours)
	maximumDemand = np.random.uniform(nominalDemand, maxNominalDemand+1, numOfHours)
	preferenceSatisfaction = np.random.uniform(0.1, 1, numOfHours)
	LAs.append(Aggregator(numOfHours, nominalDemand, minimumDemand, maximumDemand, maxPriceReference, preferenceSatisfaction))


In [677]:
# # print all the values for each LA
# for i in range(numOfLA):
# 	print("LA", i)
# 	print("nominal demand: ", LAs[i].nominalDemand)
# 	print("minimum demand: ", LAs[i].minimumDemand)
# 	print("maximum demand: ", LAs[i].maximumDemand)
# 	print("max price reference: ", LAs[i].maxPriceReference)
# 	print("preference satisfaction: ", LAs[i].preferenceSatisfaction)

In [678]:
marginalCost = np.random.randint(1, 1000, numOfHours)
unitSalePrice = np.random.randint(1, 100, numOfHours)
dso = DSO(numOfHours, marginalCost, unitSalePrice, maxPriceReference)

In [679]:
# append all the LAs to the DSO
for i in range(numOfLA):
	dso.addAggregator(LAs[i])

In [680]:
# # print all the values for the DSO
# print("DSO")
# print("marginal cost: ", dso.marginalCost)
# print("unit sale price: ", dso.unitSalePrice)


In [681]:
dso.generateUnitSalePrice()

In [682]:
# print unit sale price for the DSO
print("unit sale price: ", dso.unitSalePrice)

unit sale price:  [ 98 238 189 114 277 191 209 104 117 177 112 205 101 165 102 135 263 276
 185 250 215 273 179  83  96 222 156 219 134 180 156 107 218 179 147 213
 230 259 132 152 279 162 270 211 183 197 203 256 129 104 150  63 154 134
  87  84 188  58 229 245 232 222 276 217 146 126 197 209 166 129  85 236
  32 254 127 224 102 271 232 224 167 275 210 249 237 246 151  64 123 169
  80 252 209 238 266 177 107 104 190 128 128 160 141 174 245 261 102 189
  81 237 236 280  81 142 174 178 148 186 224  85 220  77 230 124  95  28
 260 256  83 149  77 125 261 253 203 237 162 259 123 140  97  90 124 123
  86 210  37  84 130 158 137 154 103 210 262 273 254 185  77 159  69 220
 164 150 117 241 104 101 180 202 276 271 253 235 260  78 269 259 237 260
  51 155 236 165 266 155  98 208 275  93  44 149 148 104 207 115 171 209
 120 118 212 178 264 244 136  48 175 132 120  69 133 108 172 231 135 172
 279 167  38 135  94 248 150 231 168 219  62 233 132  85  97  43 101 127
  57 204 172 122 132 116 201  79 

In [683]:
dso.maxPriceReference

281

In [684]:
LAs[0].preferenceSatisfaction

array([0.99275134, 0.44196325, 0.98698323, 0.78090364, 0.89401845,
       0.23302976, 0.39147071, 0.72753273, 0.5206586 , 0.13060772,
       0.36634813, 0.69245993, 0.83877738, 0.9710941 , 0.26383052,
       0.34263986, 0.37803637, 0.55590633, 0.41569455, 0.28409719,
       0.31920882, 0.93520847, 0.45421701, 0.76951106, 0.3350913 ,
       0.88390026, 0.74392019, 0.75308104, 0.18122061, 0.20963385,
       0.68350004, 0.93932741, 0.15598574, 0.93119377, 0.99655217,
       0.23420223, 0.61788249, 0.72387298, 0.51046116, 0.15576378,
       0.67057335, 0.76799333, 0.88376438, 0.45911441, 0.81978317,
       0.34097157, 0.75812003, 0.46583846, 0.12913124, 0.68460143,
       0.61378776, 0.24395832, 0.70372572, 0.13776049, 0.26480815,
       0.66911102, 0.18677134, 0.75008263, 0.88099375, 0.98125811,
       0.23639388, 0.44182519, 0.66083691, 0.47493564, 0.95288633,
       0.26399321, 0.67420604, 0.52430008, 0.70969934, 0.13800692,
       0.71144045, 0.98204629, 0.74954678, 0.55109242, 0.43130

In [685]:
# print(dso.aggregators)

In [686]:
# DSO gives the price for each hour & each LA calculates the optimal DR
for i in range(numOfLA):
	# la = LAs[i]
	la = dso.aggregators[i]
	la.optimalDR(dso)

In [687]:
# print best load response for each LA
for i in range(numOfLA):
	print("LA", i)
	print("best load response: ", LAs[i].bestLoadResponse)

LA 0
best load response:  [0.3982724317411197, -0.5278566406847179, 0.12486345526750128, 0.250742211382907, -0.03299563140468349, -1.4655860909783263, -0.8146163490750526, 0.521583130372871, 0.34053425865273984, -7.680547932649503, -0.12136931682558441, -0.04847482563821762, 0.35143929957118974, 0.17432357969973275, -0.7986454379823327, -0.7132747539690155, -1.7373040409961185, -0.7905898172894787, -0.8883492681537665, -2.304439203222795, -1.3082428872947518, -0.016365289184253406, -0.2625314904945537, 0.4397467759863824, -0.019274168337388313, 0.04234848686013068, 0.1244351180929348, -0.012937309584429743, -1.5634162552581559, -1.6702992796081397, 0.13199525031620973, 0.3963877688555236, -6.108008805548599, 0.205601932502763, 0.3369635198354143, -2.299559308581903, -0.20317808349681504, -0.18113125351887674, 0.085080424997727, -2.939735747523629, -0.20471592962060303, 0.2051920322224904, -0.04793021018374925, -0.468854277706802, 0.13730864607102444, -0.9579117466019088, 0.026321279657

In [688]:
# print best load response for each LA
for i in range(numOfLA):
	print("LA", i)
	print("best load response: ", dso.aggregators[i].bestLoadResponse)

LA 0
best load response:  [0.3982724317411197, -0.5278566406847179, 0.12486345526750128, 0.250742211382907, -0.03299563140468349, -1.4655860909783263, -0.8146163490750526, 0.521583130372871, 0.34053425865273984, -7.680547932649503, -0.12136931682558441, -0.04847482563821762, 0.35143929957118974, 0.17432357969973275, -0.7986454379823327, -0.7132747539690155, -1.7373040409961185, -0.7905898172894787, -0.8883492681537665, -2.304439203222795, -1.3082428872947518, -0.016365289184253406, -0.2625314904945537, 0.4397467759863824, -0.019274168337388313, 0.04234848686013068, 0.1244351180929348, -0.012937309584429743, -1.5634162552581559, -1.6702992796081397, 0.13199525031620973, 0.3963877688555236, -6.108008805548599, 0.205601932502763, 0.3369635198354143, -2.299559308581903, -0.20317808349681504, -0.18113125351887674, 0.085080424997727, -2.939735747523629, -0.20471592962060303, 0.2051920322224904, -0.04793021018374925, -0.468854277706802, 0.13730864607102444, -0.9579117466019088, 0.026321279657

In [689]:
# DSO takes back the load response from each aggregator
m = np.random.randint(1, 100)
# theta = np.random.uniform(0.1, 1)
# omega = np.random.uniform(0.1, 1)
theta = 0.01
omega = 5
dso.maxUtility = dso.utilityFunction(m, theta, omega)
dso.maxUtility

23118.366438504334

In [690]:
# print utility for each house
for i in range(numOfLA):
	print("LA", i)
	ut = LAs[i].utilityFunction(dso)
	print("utility: ", ut)

LA 0
utility:  -17068.110743543322
LA 1
utility:  -117796.56395194701
LA 2
utility:  35110.826782457916


In [691]:
dso.getPAR()

-3242.4339061045935

Stackelberg Game

In [692]:
# import gameanalysis
# import numpy as np

# # Define the DSO and aggregator objects
# totalHours = 24
# marginalCost = np.random.rand(totalHours)
# unitSalePrice = np.random.rand(totalHours)
# maxPriceReference = 1
# preferenceSatisfaction = np.random.rand(totalHours)
# actualLoad = np.random.rand(totalHours)
# nominalDemand = np.random.rand(totalHours)
# minimumDemand = np.random.rand(totalHours)
# maximumDemand = np.random.rand(totalHours)

# dso = gameanalysis.StackelbergGame(DSO.numberOfAggregators)
# for i in range(DSO.numberOfAggregators):
#     agg = DSO.aggregators[i]
#     u = np.zeros((totalHours, totalHours))
#     for j in range(totalHours):
#         u[j, :] = dso.devfn(DSO.utilityFunction, agg, j)
#     dso.add_player(u)

# # Calculate the equilibrium strategy for the DSO
# dso_eq = dso.solve()[0]

# # Calculate the equilibrium strategies for each aggregator
# for i in range(DSO.numberOfAggregators):
#     agg = DSO.aggregators[i]
#     agg_strategies = []
#     for j in range(totalHours):
#         s = dso_eq.strategies[agg.index][j, :]
#         agg_strategies.append(s)
#     agg_eq = gameanalysis.regret.mixture_regret(
#         agg_strategies, DSO.aggregators[i].utilityFunction)


In [693]:
import nashpy as nash

In [695]:
# game = nash.Game(dso.utilityFunction, dso.utilityFunction)